In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import export_graphviz
import pydot

### 1. Introduction 

In this notebook, we will look at how random forest works by analyzing the [NOAA Climate Data online tool](https://www.ncdc.noaa.gov/cdo-web/) for Seattle, WA.  Obviously we could just watch the Weather Channel, but we will pretend that it doesn't exist and we want to predict it using our random forest model.  Since our output is the next day's temperature, which is a continuous value, and since we have a dataset containing output labels associated with our features, this is a supervised regression problem. 

### 2. Workflow 

### 3. Dataset

This dataset consists of NOAA weather data from Seattle, Washington over the 2016 calendar year.  

In [73]:
# Load dataset
df = pd.read_csv('temps.csv')
df.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41


We won't be using `forecast_noaa`, `forecast_acc`, or `forecast_under`, so let's go ahead and get rid of those: 

In [74]:
# Remove forecast_noaa, forecast_acc, and forecast_under
df.drop(['forecast_noaa', 'forecast_acc', 'forecast_under'], axis=1, inplace=True)
df.head()

,year,month,day,week,temp_2,temp_1,average,actual,friend
0,2016,1,1,Fri,45,45,45.6,45,29
1,2016,1,2,Sat,44,45,45.7,44,61
2,2016,1,3,Sun,45,44,45.8,41,56
3,2016,1,4,Mon,44,41,45.9,40,53
4,2016,1,5,Tues,41,40,46.0,44,41


In the DataFrame, we have the following columns:

- **year**: All data points are days within 2016
- **month**: Integer number for the month of each year
- **day**: Integer number for the day of each month
- **week**: The day of the week as an abbreviated string
- **temp_2**: The maximum temperature two days prior
- **temp_1**: The maximum temperature one day prior
- **average**: The historical average maximum temperature on that day
- **actual**: The actual maximum temperature measurement
- **friend**: A friend's prediction, a random number ranging from 20 below the average to 20 above the average

### 4. Outliers/Missing Data

Before proceeding any further into machine learning models, let's check if there are any outliers, missing values, or strange observations in the data.  First, let's check the shape:

In [75]:
# Shape of data
df.shape

(348, 9)

With 366 days in 2016, we are missing 18 days.  If one looks through what is on the NOAA website, it is clear that these days were simply not recorded.  This is a only a small portion of our total number of days, and the pattern appears to be random, so we will not worry about it.  To check for outliers, let's look at the summary statistics:

In [76]:
# Summary statistics
df.describe()

,year,month,day,temp_2,temp_1,average,actual,friend
count,348.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000
mean,2016.0,6.477011,15.514368,62.652299,62.701149,59.760632,62.543103,60.034483
std,0.0,3.498380,8.772982,12.165398,12.120542,10.527306,11.794146,15.626179
min,2016.0,1.000000,1.000000,35.000000,35.000000,45.100000,35.000000,28.000000
25%,2016.0,3.000000,8.000000,54.000000,54.000000,49.975000,54.000000,47.750000
50%,2016.0,6.000000,15.000000,62.500000,62.500000,58.200000,62.500000,60.000000
75%,2016.0,10.000000,23.000000,71.000000,71.000000,69.025000,71.000000,71.000000
max,2016.0,12.000000,31.000000,117.000000,117.000000,77.400000,92.000000,95.000000


In [77]:
# Check for NA values
df.isnull().sum()

year       0
month      0
day        0
week       0
temp_2     0
temp_1     0
average    0
actual     0
friend     0
dtype: int64

Our data looks ok, although in a more thorough project, I would probably spend more time checking for outliers in this section, especially if the data was not from a mostly clean source like NOAA.  So we will proceed and conclude that there are no obvious outliers or missing value issues in the data. 

### 5. Data Preprocessing

Before we can feed this data into a model, we will have to do a bit of preprocessing.  The first step will be one-hot encoding.  Since we have days of the week as one of our features, which is categorical, we will need to one-hot encode it.  We might be tempted to just assign a value of 1-7 or something for each day, but this inadvertently leads to the algorithm placing more importance on Sunday, since it would have a higher numerical value.  One-hot encoding takes a categorical feature with n classes and creates n new features in the dataset where each feature representing a parituclar class assigns a 1 to rows of the class and 0 to everything else.  So we end up adding more features to our dataset, but we are able to effectively represent a categorical feature.  

In [78]:
# One-hot encode date
df = pd.get_dummies(df)
df.head()

,year,month,day,temp_2,temp_1,average,actual,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2016,1,1,45,45,45.6,45,29,1,0,0,0,0,0,0
1,2016,1,2,44,45,45.7,44,61,0,0,1,0,0,0,0
2,2016,1,3,45,44,45.8,41,56,0,0,0,1,0,0,0
3,2016,1,4,44,41,45.9,40,53,0,1,0,0,0,0,0
4,2016,1,5,41,40,46.0,44,41,0,0,0,0,0,1,0


In [79]:
df.shape

(348, 15)

So our dataset now has 15 features instead of 9 and all columns are in numerical format. Let's now split the dataset into our target and features:

In [89]:
# Separate features and target
y = df['actual']
X = df.drop('actual', axis = 1)

# Saving feature names for later use
feature_list = list(X.columns)

Let's also now split the X and y into training and test sets using a size of 0.75:

In [81]:
# Split into training and test sets with 25% validation size
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

### 6. Establish a Baseline

Before we implement our model, we want to first establish a baseline, which gives us a check on whether our model is beating a mediocre guess or not.  In this case, we can use the historical max temperature averages.  So this baseline is essentially the error we would get if we just predicted the average maximum temperature for all days.  

In [83]:
# Baseline predictions = historical averages
baseline_preds = X_test['average']
baseline_errors = abs(baseline_preds - y_test)
print(f'Average baseline error: {round(np.mean(baseline_errors), 2)}')

Average baseline error: 5.06


So our baseline is around 5.06 degrees.  If we cannot beat that, our model is clearly not working as we hoped. 

### 7. Train Model 

Now we are ready to go ahead and train our model with the final dataset:

In [84]:
# Train random forest regressor with 1000 decision trees on our data
rf = RandomForestRegressor(n_estimators=1000, random_state=1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=1)

### 8. Make Predictions on Test Set

When performing regression, MSE and RMSE are often used as error metrics.  In our case, we will use mean absolute error, since we are interested in how far away our average prediction is from the actual value.  So let's compare our predictions with the real data:

In [86]:
# Predict values 
predictions = rf.predict(X_test)

# Calculate absolute errors
errors = abs(predictions - y_test)
print(f'Mean absolute error: {round(np.mean(errors), 2)}')

Mean absolute error: 3.86


So our average estimate is off by 3.86 degrees!  That is at least over a 1 degree improvement from the baseline.  This is over a 25% improvement from the baseline, so depending on the problem, this could be significant.  

### 9. Determine Performance Metrics 

Let's understand our predictions in a bigger context by calculating an average using mean average percentage error subtracted from 100%: 

In [87]:
# Mean average percentage error
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print(f'Accuracy: {round(accuracy, 2)}')

Accuracy: 93.95


That's not too bad! This model predicts the next day's temperature to almost 94% accuracy.  

### 10. Model Interpretation 

Random forest is often regarded as a black box.  It is a complicated algorithm capable of detecting very complex patterns in data, which makes it a bit harder to dissect than something like simple linear regression.  To get under the hood of a random forest model, we can do two things: look at a single tree in the forest, and look at feature importances.  

Starting with examining trees, we can look at any tree out of the many trees in our random forest model.  Here's an example

In [90]:
# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

The next thing we can do is look at feature importances.  The feature importances are relative in the sense that they represent how much including a particular variable improves the prediction.  

In [91]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance 
                          in zip(feature_list, importances)]

# Sort the feature importances by most important
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse=True)

# Print feature and importance
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: temp_1               Importance: 0.68
Variable: average              Importance: 0.21
Variable: temp_2               Importance: 0.03
Variable: friend               Importance: 0.03
Variable: day                  Importance: 0.02
Variable: month                Importance: 0.01
Variable: year                 Importance: 0.0
Variable: week_Fri             Importance: 0.0
Variable: week_Mon             Importance: 0.0
Variable: week_Sat             Importance: 0.0
Variable: week_Sun             Importance: 0.0
Variable: week_Thurs           Importance: 0.0
Variable: week_Tues            Importance: 0.0
Variable: week_Wed             Importance: 0.0


We see based off this that overwhelmingly, the most important predictor is `temp_1`, the max temperature the day before.  This makes sense, since the previous day's temperature is generally indicative of what tomorrow's temperature will be like.  The second most important feature was the historical average.  This makes sense also.  The days of the weak and the year have no importance at all, which also makes sense because we would not generally believe that a particular day is prone to a certain type of weather pattern than another day.  It is important to note, however, that much like the previous day in weather data is important, if we were looking at **climate** data on much longer time scales, we would NOT expect the year to have no importance, and in fact, much like `temp_1` in this, it would probably be the most important on longer time scales.  

In future verions of this model, we can remove the unimportant features and the importance probably will not suffer.  In certain other models, such as SVM, we would use the RF feature importance values as a feature selection.  

In [100]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)

# Extract the two most important features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = X_train[['temp_1', 'average']]
test_important = X_test[['temp_1', 'average']]

# Train the random forest
rf_most_important.fit(train_important, y_train)

# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - y_test)

# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 3.92 degrees.
Accuracy: 93.76 %.


Wow! We could've used ONLY two features and gotten almost the exact same accuracy as before.  So in conclusion, this was a basic implementation of the random forest algorithm in Python.  How could we improve this? We could try hyperparameter optimization, a different algorithm, or collect more data. 